<a href="https://colab.research.google.com/github/isabelklint/scrapers/blob/main/Ethiopian_reporter_WEB_SCRAPER_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://github.com/tilburgsciencehub/data-ranking-the-brands/blob/main/webscraping_ranking_the_brands.ipynb
!pip install BeautifulSoup4
!pip install pandas
!pip install requests

from bs4 import BeautifulSoup
import requests
import pandas as pd


In [ ]:
soups = []
# categories do not include drop-downs
# categories = ['politics', 'business', 'editorial', 'kibur-minister', 'sport', 'kinin-and-bahil', 'opinion', 'temuaget', 'speak-your-mind', 'world' ]
# https://www.ethiopianreporter.com
page_link = 'https://www.ethiopianreporter.com/news/'
request_object = requests.get(page_link)
source_code = request_object.text
soup = BeautifulSoup(source_code, "lxml")
soups.append(soup)


In [ ]:
# make some lists
list_of_URLS = []
scrapedURLS = []


In [ ]:
# Find all the <a> tags with href containing the word "article"
list_of_URLS = [a['href'] for a in soup.find_all('a', href=lambda href: href and 'article' in href)]

# Print the article links
# print(list_of_URLS)
    

[]


In [ ]:
# print(soups)

In [ ]:
# get a list of urls that make sense for news items in amaric
# these won't include videos tho
# video url: https://mereja.com/video2/watch.php?vid=f94162ab5
for url in list_of_URLS:
  if url.startswith('https://www.bbc.com/amharic/articles/') == True:
    scrapedURLS.append(url)
  elif url.startswith('/amharic/articles/'):
    scrapedURLS.append('https://www.bbc.com' + url)
  

In [ ]:
# how many urls?
len(scrapedURLS)


0

In [ ]:
# get rid of duplicates
unique_list = list(set(scrapedURLS))
print(len(unique_list))


0


In [ ]:
# set name again
scrapedURLS = unique_list


In [ ]:
# an example url
print(scrapedURLS[0])


https://www.bbc.com/amharic/articles/c9r007rqdgzo


In [ ]:
# adapted from https://blog.devgenius.io/use-selenium-webdriver-in-google-colab-d5f2dba1d9f5
#First, we install cloudscraper and googletrans modules that we are going to use for scraping and translating the content.

!pip install cloudscraper
!pip install google_trans_new


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#With cloudscraper we scrape the URL and with Beutiful Soup we parse the URLs.
#We put everything together in order based on importance.
import cloudscraper
from bs4 import BeautifulSoup


In [ ]:
# declare data as an empty DataFrame
data = pd.DataFrame(columns = ['Date', 'Title', 'URL'])

def scrape_this_url(url):
    scraper = cloudscraper.create_scraper() 
    headers = {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}
    
    try:
        r = scraper.get(url, headers=headers)
        # soup = BeautifulSoup(r.content, 'html.parser')
        soup = BeautifulSoup(r.content, 'lxml')
        # get title
        # <meta data-react-helmet="true" property="og:title" content="በዩክሬኑ ጦርነት ሩሲያ ከሰራዊቷ ይልቅ ሰላይዎቿ የበለጠ ውጤታማ እንደሆኑ ተነገረ  - BBC News አማርኛ">                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 "url":"https://news.files.bbci.co.uk/ws/img/logos/og/amharic.png"},"noBylinesPolicy":"https://www.bbc.com/amharic/institutional-49283133#authorexpertise"}}]}
        # get date
        # <meta data-react-helmet="true" name="article:published_time" content="2023-03-30T04:07:16.525Z">
        a_title = soup.find("meta", property = "og:title")
        # a_date = soup.find("meta", name = "article:published_time")
        title = a_title["content"] if a_title else ""
        for tag in soup.find_all("meta"):
            if tag.get("name", None) == "article:published_time":
                date = tag.get("content", None)

        # Create a dictionary with the data to add
        data_dict = {'Date': date, 'Title': title, 'URL': url}

        # return the dictionary
        return data_dict

    except Exception as e:
        print(e)
        return {}


In [ ]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


In [ ]:
# loop through the list of URLs and append the data to the DataFrame
for url in scrapedURLS:
    data_dict = scrape_this_url(url)
    if data_dict:
        data = data.append(data_dict, ignore_index=True)


In [ ]:
data.head()


,Date,Title,URL
0,2023-03-29T08:41:55.863Z,ትንሿ ሌሶቶ በሰፊ የደቡብ አፍሪካ ግዛቶች ላይ የይገባኛል ጥያቄ ልታቀርብ...,https://www.bbc.com/amharic/articles/c9r007rqdgzo
1,2023-03-31T04:07:15.673Z,የአላማጣ ከተማ ወቅታዊ ሁኔታ እና የነዋሪዎች ቅሬታ - BBC News አማርኛ,https://www.bbc.com/amharic/articles/ckkqqexrzv0o
2,2023-04-02T05:09:29.676Z,ቤተሰቦቼን ማግኘት አለመቻሌ ከፍተኛ ፈተና ሆኖብኝ ነበር : ጉዳፍ ፀጋይ ...,https://www.bbc.com/amharic/articles/cpvqxkqdqpzo
3,2023-03-31T10:43:49.695Z,“ኡጋንዳውያኑ ወደ ኢትዮጵያ የገቡት የምጽዓት ቀንን ሽሽት አይደለም” የኛ...,https://www.bbc.com/amharic/articles/cq5ew7xnpgwo
4,2023-03-27T14:05:18.674Z,ለረጅም ጊዜ ከዕይታ ተሰውሮ የቆየው የአሊባባ መሥራች ጃክ ማ በይፋ ታየ ...,https://www.bbc.com/amharic/articles/c6pl6m72863o


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    60 non-null     object
 1   Title   60 non-null     object
 2   URL     60 non-null     object
dtypes: object(3)
memory usage: 1.5+ KB
